# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty')
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period')
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit')

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [1]:
!pip install sklearn-crfsuite==0.3.6
!pip install nltk==3.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.6 MB/s eta 0:00:00


#### Import the necessary libraries

In [2]:
import nltk
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split
import pickle
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [3]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [4]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s):
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [5]:
def tuples_maker(inp, out):
  # Input validation
    if len(input_sigs) != len(output_labels):
        raise ValueError("Input signatures and output labels must have the same length")

    # Initialize result list
    sample_data = []

    # Process each prescription
    for tokens, labels in zip(input_sigs, output_labels):
        # Validate that tokens and labels have same length
        if len(tokens) != len(labels):
            raise ValueError(f"Mismatch in tokens and labels length: {tokens} vs {labels}")

        # Create tuples for current prescription
        prescription_tuples = []
        for token, label in zip(tokens, labels):
            prescription_tuples.append((token, label))

        # Add to result
        sample_data.append(prescription_tuples)


    return sample_data

In [6]:
from pprint import pprint

# Create the tuples using our function
tuples_maker_output = tuples_maker(input_sigs, output_labels)

# Print in a nicely formatted way
pprint(tuples_maker_output)

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output:
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [7]:
def triples_maker(whole_data):
    sample_data = []

    # Process each prescription
    for prescription in whole_data:
        # Extract just the tokens from the current prescription
        tokens = [token for token, _ in prescription]

        # Get POS tags for the tokens
        pos_tags = nltk.pos_tag(tokens)

        # Create triples for current prescription
        prescription_triples = []
        for (token, pos_tag), (_, label) in zip(pos_tags, prescription):
            prescription_triples.append((token, pos_tag, label))

        # Add to result
        sample_data.append(prescription_triples)
    return sample_data

In [8]:
sample_data = triples_maker(tuples_maker_output)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [10]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data
- Feature extraction
- Train-test-split

In [11]:

# Feature extraction
def extract_features(doc):
    return [token_to_features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [label for token, postag, label in doc]

X = [extract_features(doc) for doc in sample_data]
y = [get_labels(doc) for doc in sample_data]

# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
"""# First, let's extract features for each sentence in our data
def extract_features_and_labels(data):
    X = []  # Features for all sentences
    y = []  # Labels for all sentences

    # Process each prescription
    for doc in data:
        # Extract features for each token in the prescription
        sentence_features = []
        sentence_labels = []

        for i in range(len(doc)):
            # Get features for current token
            token_features = token_to_features(doc, i)
            sentence_features.append(token_features)

            # Get label (it's the third element in our triple)
            sentence_labels.append(doc[i][2])

        X.append(sentence_features)
        y.append(sentence_labels)

    return X, y

# Extract features from our sample data
X, y = extract_features_and_labels(sample_data)

# Split into train and test sets (using sklearn)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Print some information about the split
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Optional: Print example of features for first token of first prescription
print("\nExample features for first token:")
if X_train:
    pprint(X_train[0][0])"""

'# First, let\'s extract features for each sentence in our data\ndef extract_features_and_labels(data):\n    X = []  # Features for all sentences\n    y = []  # Labels for all sentences\n    \n    # Process each prescription\n    for doc in data:\n        # Extract features for each token in the prescription\n        sentence_features = []\n        sentence_labels = []\n        \n        for i in range(len(doc)):\n            # Get features for current token\n            token_features = token_to_features(doc, i)\n            sentence_features.append(token_features)\n            \n            # Get label (it\'s the third element in our triple)\n            sentence_labels.append(doc[i][2])\n            \n        X.append(sentence_features)\n        y.append(sentence_labels)\n    \n    return X, y\n\n# Extract features from our sample data\nX, y = extract_features_and_labels(sample_data)\n\n# Split into train and test sets (using sklearn)\nfrom sklearn.model_selection import train_test_

### Training the CRF model with the features extracted using the feature extractor method

In [17]:
from sklearn_crfsuite import CRF


In [22]:
# Training the CRF model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,                # L1 penalty
    c2=0.01,               # L2 penalty (adjusted to match output)
    max_iterations=1000,   # Maximum iterations
    all_possible_transitions=True,
    verbose=True           # Set verbose to True for output during training
)

# Train the model
crf.fit(X_train, y_train)

# Save the trained model as a pickle file
model_filename = 'crf_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(crf, model_file)


loading training data to CRFsuite: 100%|██████████| 44/44 [00:00<00:00, 10829.09it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1841
Seconds required: 0.006

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.00  loss=598.93   active=1828  feature_norm=1.00
Iter 2   time=0.01  loss=326.10   active=1568  feature_norm=6.31
Iter 3   time=0.00  loss=256.46   active=1435  feature_norm=7.88
Iter 4   time=0.00  loss=216.98   active=1338  feature_norm=8.21
Iter 5   time=0.01  loss=154.74   active=1137  feature_norm=9.44
Iter 6   time=0.00  loss=113.99   active=1118  feature_norm=10.83
Iter 7   time=0.00  loss=80.21    active=1050  feature_norm=14.53
Iter 8   time=0.00  loss=63.27    active=1088  feature_norm=16.00
Iter 9   time=0.00  loss=58.92    active=1066  feature_norm=16.56
Iter 10  t

In [ ]:


# Submit training data to the trainer


# Set the parameters of the model


# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1971
Seconds required: 0.010

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 734.228583
Feature norm: 1.000000
Error norm: 148.137282
Active features: 1948
Line search trials: 1
Line search step: 0.005207
Seconds required for this iteration: 0.001

***** Iteration #2 *****
Loss: 457.872349
Feature norm: 5.727530
Error norm: 185.004580
Active features: 1861
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 331.095715
Feature norm: 7.055854
Error norm: 87.348079
Active features: 1853
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration:

***** Iteration #150 *****
Loss: 47.261611
Feature norm: 23.027520
Error norm: 0.046637
Active features: 295
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.001

***** Iteration #151 *****
Loss: 47.261590
Feature norm: 23.026657
Error norm: 0.053513
Active features: 295
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.001

***** Iteration #152 *****
Loss: 47.261530
Feature norm: 23.026726
Error norm: 0.045319
Active features: 295
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.001

***** Iteration #153 *****
Loss: 47.261508
Feature norm: 23.025995
Error norm: 0.048871
Active features: 295
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.001

***** Iteration #154 *****
Loss: 47.261456
Feature norm: 23.026147
Error norm: 0.040011
Active features: 295
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteratio

### Predicting the test data with the built model

In [25]:
# Load the trained CRF model
model_filename = 'crf_model.pkl'
with open(model_filename, 'rb') as model_file:
    crf = pickle.load(model_file)

In [30]:
import nltk

# Download the necessary NLTK data
nltk.download('punkt')

# Tokenize the input sentence
sig = "take 2 tabs every 6 hours x 10 days"
tokens = nltk.word_tokenize(sig)
print(tokens)

['take', '2', 'tabs', 'every', '6', 'hours', 'x', '10', 'days']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
# Get POS tags for the tokens
pos_tags = nltk.pos_tag(tokens)
print(pos_tags)

[('take', 'VB'), ('2', 'CD'), ('tabs', 'NNS'), ('every', 'DT'), ('6', 'CD'), ('hours', 'NNS'), ('x', 'RB'), ('10', 'CD'), ('days', 'NNS')]


In [32]:
# Create features for the input sentence
input_data = [(token, pos_tag) for token, pos_tag in pos_tags]
X_test = [token_to_features(input_data, i) for i in range(len(input_data))]
print(X_test)

[['bias', 'word.lower=take', 'word[-3:]=ake', 'word[-2:]=ke', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=VB', 'BOS', '+1:word.lower=2', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=True', '+1:postag=CD'], ['bias', 'word.lower=2', 'word[-3:]=2', 'word[-2:]=2', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=True', 'postag=CD', '-1:word.lower=take', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=VB', '+1:word.lower=tabs', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNS'], ['bias', 'word.lower=tabs', 'word[-3:]=abs', 'word[-2:]=bs', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=NNS', '-1:word.lower=2', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=True', '-1:postag=CD', '+1:word.lower=every', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=DT'], ['bias', 'word.l

In [33]:
# Predict labels using the loaded model
y_pred = crf.predict_single(X_test)
print(y_pred)

['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']


### Putting all the prediction logic inside a predict method

In [38]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    # Load the trained CRF model
    model_filename = 'crf_model.pkl'
    with open(model_filename, 'rb') as model_file:
        crf = pickle.load(model_file)

    # Tokenize the input sentence
    tokens = nltk.word_tokenize(sig)

    # Get POS tags for the tokens
    pos_tags = nltk.pos_tag(tokens)

    # Create features for the input sentence
    input_data = [(token, pos_tag) for token, pos_tag in pos_tags]
    X_test = [token_to_features(input_data, i) for i in range(len(input_data))]

    # Predict labels using the loaded model
    y_pred = crf.predict_single(X_test)

    return print(sig, "\n", y_pred)


### Sample predictions

In [ ]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'PeriodUnit', 'AT', 'WHEN']]


In [ ]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [ ]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]


In [ ]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
predictions = predict("x 30 days")

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [ ]:
predictions = predict("x 20 months")

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [ ]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [ ]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Frequency', 'TID']]


In [ ]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']]


In [ ]:
predictions = predict("inject 3 units orally")

inject 3 units orally
[['Method', 'Qty', 'Form', 'PO']]


In [ ]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
[['PO', 'Method', 'Qty', 'Form', 'TID']]


In [ ]:
predictions = predict("by mouth take three caps")

by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [ ]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [ ]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [ ]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'BID', 'PO', 'AT', 'WHEN']]


In [ ]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [ ]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'AND', 'EVERY', 'Period', 'PeriodUnit']]


In [ ]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
[['WHEN', 'QID', 'Qty', 'Form']]


In [ ]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'TID', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']]


In [ ]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
[['Method', 'Qty', 'Form']]


In [ ]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [ ]:
prediction = predict("take two to four tabs")

take two to four tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [ ]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("every 60 mins")

every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [ ]:
prediction = predict("every 10 mins")

every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [ ]:
prediction = predict("every two to four months")

every two to four months
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [ ]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'DurationMax', 'DurationUnit', 'Method', 'Qty', 'Form']]


In [ ]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
[['FOR', 'Duration', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [ ]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit']]


In [ ]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
[['PO', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]


In [ ]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [ ]:
prediction = predict("10 tabs")

10 tabs
[['Qty', 'Form']]
